In [2]:
import logging
from typing import Callable, Literal, Optional, Union

import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.optimize import minimize

from bwlogger import StyleAdapter
import argparse
import logging
from pathlib import Path
from typing import Literal, Optional, Union

import numpy as np
import pandas as pd

from bwbbgdl import GoGet
from bwlogger import StyleAdapter, basic_setup
from bwutils import TODAY, Date

from utils import calculate_weights, _exponentially_decaying_weights
from plot import plot_results

c:\Users\pcampos\Anaconda3\envs\bwgi\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [23]:
FX_TRACKER_DICT = {
    x: f"JPFCT{x} Index"
    for x in [
        "AED",
        "ARS",
        "BRL",
        "CLP",
        "CNY",
        "COP",
        "CZK",
        "HUF",
        "IDR",
        "INR",
        "MXN",
        "MYR",
        "PEN",
        "PHP",
        "PLN",
        "RON",
        "RUB",
        "SAR",
        "THB",
        "TRY",
        "ZAR",
    ]
}

In [125]:
# these are in ER in USD
EM_CDS_TRACKER_DICT = {
    "BRL": "GSCDBRBE Index",
    "CNY": "GSCDCHBE Index",
    "MXN": "GSCDMEBE Index",
    "ZAR": "GSCDSOBE Index",
}

# these are in LOC ER with pnl converted to USD
IRS_TRACKER_DICT = {
    "BRL": "GSSWBRN5 Index",
    "CNY": "GSSWCNN5 Index",
    "MXN": "GSSWMXN5 Index",
    "ZAR": "GSSWZAN5 Index",
}

EQ_TRACKER_DICT = {
    "BRL": "BNPIFBR Index",  # in BRL
    "CNY": "BNPIFCNO Index",  # China onshore but with pnl converted to USD
    "ZAR": "BNPIFSA Index",  # in ZAR
    # "MXN": "???? Index",
}

# these are in ER in USD
# FX_TRACKER_DICT = {x: f"JPFCT{x} Index" for x in EM_CDS_TRACKER_DICT}


# def country_assets_portfolio(
#     ccy: str,
#     vol_target: float = 0.1,
#     dt_ini: Date = "1990-12-31",
#     dt_end: Date = TODAY,
# ) -> pd.DataFrame:
mapper_ticker = FX_TRACKER_DICT
vol_target: float = 0.1
dt_ini: Date = "1990-12-31"
dt_end: Date = TODAY
method_weights = "IV"


inverse_mapper_ticker = {v: k for k, v in mapper_ticker.items()}
tickers = list(mapper_ticker.values())

g = GoGet(enforce_strict_matching=True)

tracker_df: pd.DataFrame = g.fetch(
    tickers=tickers,
    fields="PX_LAST",
    dt_ini=dt_ini,
    dt_end=dt_end,
)

tracker_df = tracker_df.pivot_table(index="date", columns="id")
tracker_df.columns = tracker_df.columns.droplevel(0)
tracker_df = tracker_df.rename(columns=inverse_mapper_ticker)#.dropna()

backtest = pd.Series(index=tracker_df.index[100:])
start_backtest = backtest.index.min()
backtest.iloc[0] = 100.0

avaialbe_trackers = get_available_trackers(tracker_df.iloc[:100])
cov = np.log(tracker_df.iloc[:100][avaialbe_trackers]).diff(21).cov() * 12.0
w = calculate_weights(method=method_weights, cov_matrix=cov)
adj_factor = vol_target / np.sqrt(w @ cov @ w)
w = adj_factor * w

weights_rebal = []
q = backtest.iloc[0] * w / tracker_df.loc[start_backtest]
s_rebal = q.copy()
s_rebal.name = start_backtest
weights_rebal.append(s_rebal)

for t, tm1 in zip(backtest.index[1:], backtest.index[:-1]):
    pnl = ((tracker_df.loc[t] - tracker_df.loc[tm1]) * q).sum()
    backtest[t] = backtest[tm1] + pnl
    if t.month != tm1.month:
        if tracker_df.loc[:t].shape[0] > 252:
            avaialbe_trackers = get_available_trackers(tracker_df.loc[:tm1])
            cov = np.log(tracker_df.loc[:tm1][avaialbe_trackers]).diff(21).cov() * 12.0
        w = calculate_weights(method=method_weights, cov_matrix=cov)
        adj_factor = vol_target / np.sqrt(w @ cov @ w)
        w = adj_factor * w
        q = backtest[tm1] * w / tracker_df.loc[tm1]
        s_rebal = q.copy()
        s_rebal.name = t
        weights_rebal.append(s_rebal)


backtest = pd.concat(
    [tracker_df, backtest.to_frame("assets")],
    axis=1,
    join="outer",
    sort=True,
)
backtest
# return backtest

C:\Users\pcampos\AppData\Local\Temp\ipykernel_42860\1479633222.py:57: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  backtest = pd.Series(index=tracker_df.index[100:])


,BRL,CLP,CNY,COP,CZK,HUF,IDR,INR,MXN,MYR,PLN,RUB,THB,TRY,ZAR,assets
date,,,,,,,,,,,,,,,,
1999-01-04,100.000,100.000,NaN,NaN,100.000,100.000,NaN,NaN,100.000,NaN,NaN,NaN,NaN,NaN,100.000,NaN
1999-01-05,100.007,100.002,NaN,NaN,99.997,99.958,NaN,NaN,100.041,NaN,NaN,NaN,NaN,NaN,100.054,NaN
1999-01-06,100.011,100.026,NaN,NaN,99.973,99.922,NaN,NaN,100.073,NaN,NaN,NaN,NaN,NaN,100.202,NaN
1999-01-07,99.939,99.976,NaN,NaN,99.866,99.885,NaN,NaN,99.952,NaN,NaN,NaN,NaN,NaN,99.767,NaN
1999-01-08,99.950,100.025,NaN,NaN,99.614,99.756,NaN,NaN,100.095,NaN,NaN,NaN,NaN,NaN,99.940,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-01,285.467,92.593,108.135,141.776,130.968,158.036,245.250,149.020,231.751,180.408,160.190,167.045,150.979,188.290,146.083,165.619717
2024-07-02,281.785,91.991,108.080,141.776,130.732,158.075,244.865,148.802,234.234,180.113,160.358,165.058,150.659,189.218,143.238,165.102092
2024-07-03,286.142,93.049,108.045,143.092,131.632,159.079,244.591,148.779,235.295,180.080,161.736,164.187,150.578,189.736,145.401,166.662885


In [138]:
def get_available_trackers(df: pd.DataFrame, min_data_points: int = 100) -> None:
    s_data_points = (~df.isna()).sum()
    filt = s_data_points >= min_data_points
    return s_data_points[filt].index


get_available_trackers(tracker_df)

Index(['BRL', 'CLP', 'CNY', 'COP', 'CZK', 'HUF', 'IDR', 'INR', 'MXN', 'MYR',
       'PLN', 'RUB', 'THB', 'TRY', 'ZAR'],
      dtype='object', name='id')

In [127]:
pd.concat(weights_rebal, axis=1).T

id,BRL,CLP,CNY,COP,CZK,HUF,IDR,INR,MXN,MYR,PLN,RUB,THB,TRY,ZAR
1999-06-01,0.021795,0.526798,NaN,NaN,0.209443,0.426429,NaN,NaN,0.359626,NaN,NaN,NaN,NaN,NaN,0.474113
1999-07-01,0.021685,0.559765,NaN,NaN,0.208861,0.436991,NaN,NaN,0.347028,NaN,NaN,NaN,NaN,NaN,0.465496
1999-08-02,0.022924,0.566503,NaN,NaN,0.207886,0.435287,NaN,NaN,0.348619,NaN,NaN,NaN,NaN,NaN,0.484050
1999-09-01,0.024133,0.573047,NaN,NaN,0.212422,0.442261,NaN,NaN,0.346327,NaN,NaN,NaN,NaN,NaN,0.479534
1999-10-01,0.024412,0.592811,NaN,NaN,0.207687,0.447366,NaN,NaN,0.344316,NaN,NaN,NaN,NaN,NaN,0.476158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-01,0.059252,0.411360,NaN,NaN,0.261257,0.167964,NaN,NaN,0.153752,NaN,NaN,NaN,NaN,NaN,0.140687
2024-04-02,0.059808,0.418733,NaN,NaN,0.263474,0.169521,NaN,NaN,0.150056,NaN,NaN,NaN,NaN,NaN,0.139591
2024-05-01,0.061808,0.407257,NaN,NaN,0.264586,0.169391,NaN,NaN,0.152760,NaN,NaN,NaN,NaN,NaN,0.138448
2024-06-03,0.063816,0.397055,NaN,NaN,0.260707,0.169532,NaN,NaN,0.155182,NaN,NaN,NaN,NaN,NaN,0.140843


In [20]:
def load_trackers(
    mapper_ticker: dict[str, str],
    dt_ini: Date = "1990-12-31",
    dt_end: Date = TODAY,
) -> pd.DataFrame:
    inverse_mapper_ticker = {v: k for k, v in mapper_ticker.items()}
    tickers = list(mapper_ticker.values())

    g = GoGet(enforce_strict_matching=True)
    tracker_df: pd.DataFrame = g.fetch(
        tickers=tickers,
        fields="PX_LAST",
        dt_ini=dt_ini,
        dt_end=dt_end,
    )

    tracker_df = tracker_df.pivot_table(index="date", columns="id")
    tracker_df.columns = tracker_df.columns.droplevel(0)
    tracker_df = tracker_df.rename(columns=inverse_mapper_ticker)
    return tracker_df

tracker_df = load_trackers(FX_TRACKER_DICT)


In [22]:
set(FX_TRACKER_DICT) - set(tracker_df.columns)

{'AED', 'ARS', 'PEN', 'PHP', 'RON', 'SAR'}

In [6]:
s_erc = backtest["assets"].copy()

# Trading Strategy
method_weights: Literal["ERC", "HRC", "IV",] = "ERC"
rebalancing_window = "1M" # 3M, 1M, 20D, 15D, 10D, 5D
vol_target: float = 0.1
# Covariance Matrix Parameters Estimation
return_period: dict[Literal["D", "W", "M", "Q", "Y"], int] = {"D": 21}  # period #days
return_rolling: bool = True
cov_window: Literal["expanding", "rolling"] = "expanding"  # Expanding or Rolling( and period #days)
cov_estimate_wegihts: Optional[dict[Literal["halflife", "alpha"], float]] = None

# Rebalancing Triggers
tol_by_asset: Optional[float] = None
tol_agg: Optional[float] = None



""
